In [ ]:
#Daniyal Amir
#20i-1806

In [12]:
%pip install nltk
%pip install transformers
%pip install torch


In [ ]:
import torch
device=torch.device("cuda")

In [1]:
import pandas as pd
import re


# Custom list of Urdu stop words
stop_words_urdu = [
    'ہے', 'کے', 'کا', 'کی', 'ہوتا', 'کو', 'سے', 'پر', 'ہیں', 'کا', 'کر', 'کے', 'لئے', 'سب', 'سے', 'ہوشربا', 'یہ', 'اور', 'یا', 'تک', 'کی', 'میں', 'بھی', 'آپ', 'کا', 'ہی', 'کوئی', 'یہ', 'کر', 'رہے', 'تھا', 'کر', 'لیا', 'آپ', 'کے', 'لئے', 'کر', 'لیا', 'لیکن', 'کیا'
]

# Assuming your CSV file is named 'stories.csv'
file_path = '/content/drive/MyDrive/stories.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, header=None, names=['Story'])

# Split the 'Story' column into sentences using Urdu full stop as delimiter
df['Sentences'] = df['Story'].apply(lambda x: [sentence.strip() for sentence in x.split('۔')] if isinstance(x, str) else [])

# Explode the list of sentences into separate rows
df = df.explode('Sentences')

# Remove leading and trailing whitespaces from the sentence text
df['Sentences'] = df['Sentences'].str.strip()

# Remove special characters (keeping Urdu characters)
df['Sentences'] = df['Sentences'].apply(lambda x: re.sub(r'[^\w\s۔]', '', x) if isinstance(x, str) else x)

# Remove Urdu stop words
df['Sentences'] = df['Sentences'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words_urdu]) if isinstance(x, str) else x)

# Custom list of pronouns and other words to exclude
custom_stop_words = ['اچھا', 'بہت', 'یہ', 'تم', 'میں', 'کون', 'کیا', 'میں', 'تم', 'آپ', 'ہم', 'آپ', 'وہ', 'یہ']

# Remove custom stop words
df['Sentences'] = df['Sentences'].apply(lambda x: ' '.join([word for word in x.split() if word not in custom_stop_words]) if isinstance(x, str) else x)

# Remove rows with empty sentences
df = df[df['Sentences'] != '']

# Keep only the 'Sentences' column
df = df[['Sentences']]

# Display the cleaned and stemmed DataFrame
print(df.shape)

# Save the cleaned and stemmed DataFrame to a new CSV file
file_path = 'processed_stories.csv'
df.to_csv(file_path, index=False)


(61378, 1)


In [ ]:
%pip install -U sentence-transformers


In [4]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Assuming df is your DataFrame with 'Sentences' column
sentences_array = df['Sentences'].values

# Handle NaN values by replacing them with an empty string
sentences_array = np.where(pd.isnull(sentences_array), '', sentences_array)

# Encode the sentences using the SentenceTransformer model
embeddings = model.encode(sentences_array)
print(embeddings)


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

[[ 0.04944848  0.0548213  -0.07967758 ...  0.04178321  0.04190407
  -0.06206924]
 [ 0.0524122   0.05700504 -0.02308431 ... -0.02573    -0.01387481
  -0.06959736]
 [ 0.00942501  0.0700694  -0.06927811 ...  0.00741725  0.04061584
  -0.0645033 ]
 ...
 [-0.01018992  0.09146421 -0.07382899 ...  0.00266158 -0.00087061
  -0.07457886]
 [ 0.03021406  0.04915351 -0.0702349  ...  0.02438644  0.06135729
  -0.05066692]
 [ 0.02397532  0.01559851 -0.00790647 ...  0.01040722 -0.00763508
  -0.06235005]]


In [6]:
import faiss
# Set the number of clusters for IVF (Inverted File with Flat)
nlist = 100
quantizer = faiss.IndexFlatL2(embeddings.shape[1])
index = faiss.IndexIVFFlat(quantizer, embeddings.shape[1], nlist, faiss.METRIC_L2)

# Train the index
index.train(embeddings)
index.add(embeddings)

# Save the Faiss index to a file
index_path = 'faiss_index'
faiss.write_index(index, index_path)

print(f"Faiss index saved to: {index_path}")


Faiss index saved to: faiss_index


In [5]:
!pip install faiss-cpu

In [7]:
from transformers import pipeline
from google.colab import files

# Upload the audio file
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]

# Create an ASR pipeline
asr_pipe = pipeline("automatic-speech-recognition", model="ihanif/whisper-medium-urdu")

# Transcribe the audio file
result = asr_pipe(audio_path)

# Print the transcription
print(result)


IndexError: ignored

In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np
import re

# Custom list of Urdu stop words
stop_words_urdu = [
    'ہے', 'کے', 'کا', 'کی', 'ہوتا', 'کو', 'سے', 'پر', 'ہیں', 'کا', 'کر', 'کے', 'لئے', 'سب', 'سے', 'ہوشربا', 'یہ', 'اور', 'یا', 'تک', 'کی', 'میں', 'بھی', 'آپ', 'کا', 'ہی', 'کوئی', 'یہ', 'کر', 'رہے', 'تھا', 'کر', 'لیا', 'آپ', 'کے', 'لئے', 'کر', 'لیا', 'لیکن', 'کیا'
]

# Custom list of pronouns and other words to exclude
custom_stop_words = ['اچھا', 'بہت', 'یہ', 'تم', 'میں', 'کون', 'کیا', 'میں', 'تم', 'آپ', 'ہم', 'آپ', 'وہ', 'یہ']

# Combine stop words lists
all_stop_words = stop_words_urdu + custom_stop_words

# Define a regular expression pattern to extract Urdu nouns, verbs, adverbs, and adjectives
urdu_pattern = re.compile(r'[\u0600-\u06FF]+')

# Get user input
user_input = input("Enter a text: ")
# user_input = result

# Remove special characters and keep only Urdu words
urdu_words = re.findall(urdu_pattern, user_input)

# Remove stop words, including custom stop words
filtered_words = [word for word in urdu_words if word not in all_stop_words]

# Create a sentence from the filtered words
filtered_sentence = ' '.join(filtered_words)

# Encode the filtered sentence using the SentenceTransformer model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
user_embedding = model.encode([filtered_sentence])

# Display the results
print(f"\nFiltered and encoded user input:\n{filtered_sentence}")
print(f"\nUser embedding:\n{user_embedding}")


Enter a text: کیا دوسری بیوی سے اتنی جلدی دل بھر گیا ؟ 

Filtered and encoded user input:
دوسری بیوی اتنی جلدی دل بھر گیا ؟

User embedding:
[[-2.28500813e-02  2.64520776e-02 -4.05035764e-02  8.17320868e-03
  -5.21556810e-02 -3.63115780e-02  9.71415564e-02 -3.78556876e-03
   3.41247655e-02 -4.33475263e-02  1.05419792e-01  3.94376218e-02
   3.87470126e-02  2.25343052e-02 -2.54171025e-02 -4.27182242e-02
  -3.63665037e-02 -3.30567025e-02 -3.45887654e-02 -8.44950080e-02
   2.97055580e-02 -2.22299043e-02  7.71031603e-02  1.54162887e-02
  -3.96670811e-02 -2.90735811e-02 -2.86820810e-02  9.87448636e-03
   7.34211206e-02 -4.86802496e-02  2.20797653e-03  9.25600678e-02
   2.86517888e-02 -2.58294642e-02 -2.02077981e-02  5.54882176e-03
  -1.21904407e-02 -7.52340555e-02  7.89298937e-02  8.20043236e-02
   7.66516011e-03 -1.56533360e-01  5.15057147e-02 -1.39052393e-02
  -2.56799087e-02  2.54536401e-02 -7.38704205e-02  5.59602380e-02
  -2.23091990e-03  1.59907248e-02 -6.98442310e-02 -3.24918926e-02
 

In [9]:
from sklearn.metrics.pairwise import cosine_similarity


# Retrieve all embeddings from the Faiss index
all_embeddings = index.reconstruct_n(0, index.ntotal)

# Calculate cosine distances between the user input and all embeddings
cosine_distances = 1 - cosine_similarity(user_embedding, all_embeddings)

# Set a maximum distance threshold for very close similarity
max_distance_threshold = 1.0  # Adjust as needed

# Find indices of sentences with distance below the threshold
candidate_indices = np.where(cosine_distances[0] <= max_distance_threshold)[0]

# Sort candidates by distance (ascending order)
sorted_indices = np.argsort(cosine_distances[0, candidate_indices])

# Select the top k most similar sentences
k = 5
selected_indices = candidate_indices[sorted_indices[:k]]

# Retrieve the selected sentences
selected_sentences = df['Sentences'].iloc[selected_indices].tolist()

# Display the results
print(f"\nTop {k} very similar sentences to the input text:")
for i, sentence in enumerate(selected_sentences):
    print(f"{i + 1}. {sentence}")



Top 5 very similar sentences to the input text:
1. ایسی باتیں دل گھائل کرتی
2. عیسائیوں ساری بستی جل گئی
3. بس بات بڑھ گئی درمیان بیچاری کنول بلیدان ہو گیا
4. ان جو رقم ملی آدھی گھر مرمت لگادی آدھی بینک جمع کروا دی
5. میری اس دیوانگی جلد پتا چل گیا


In [18]:
from transformers import MarianMTModel, MarianTokenizer
import torch

# Replace 'your-marianmt-model' with the name or path of a suitable MarianMT model for Urdu
model_name = 'Helsinki-NLP/opus-mt-ur-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)



# Translate user input
input_text = user_input
inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
with torch.no_grad():
    output = model.generate(**inputs)
user_input_translation = tokenizer.decode(output[0], skip_special_tokens=True)
print(f"\nMarianMT model translation (User Input): {user_input_translation}")

# Translate selected sentences to English and store translations in an array
sentence_translations = []
for sentence in selected_sentences:
    inputs = tokenizer(sentence, return_tensors="pt", max_length=512, truncation=True)
    with torch.no_grad():
        output = model.generate(**inputs)
    sentence_translation = tokenizer.decode(output[0], skip_special_tokens=True)
    sentence_translations.append(sentence_translation)

# Print the translations
for sentence, translation in zip(selected_sentences, sentence_translations):
    print(f"\nOriginal Sentence (Urdu): {sentence}\nMarianMT model translation (English): {translation}")



MarianMT model translation (User Input): Did the second wife get so excited?

Original Sentence (Urdu): ایسی باتیں دل گھائل کرتی
MarianMT model translation (English): Such things can lead to pain in the heart

Original Sentence (Urdu): عیسائیوں ساری بستی جل گئی
MarianMT model translation (English): All the Jewish communities died.

Original Sentence (Urdu): بس بات بڑھ گئی درمیان بیچاری کنول بلیدان ہو گیا
MarianMT model translation (English): Just the word went into the middle of the point that we're going to get a poor blood drop.

Original Sentence (Urdu): ان جو رقم ملی آدھی گھر مرمت لگادی آدھی بینک جمع کروا دی
MarianMT model translation (English): The sum they paid was half of the money they had paid was half of the pieces of the bank.

Original Sentence (Urdu): میری اس دیوانگی جلد پتا چل گیا
MarianMT model translation (English): My madness was quickly caught down.


In [17]:
!pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


In [27]:
import replicate
import os
# Prompts
prompt = f"This is the context. {'.'.join(sentence_translations)}\n Answer the user question: {user_input_translation} with respect to the above given context"

# Printing the result

os.environ["REPLICATE_API_TOKEN"] = "r8_DTRf1lFxRUzLDtCmE5CmpLEZEwAMpBD2pCfsL"
api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])
output = api.run(
    "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
        input={"prompt": prompt}
    )

output_array = []
for item in output:
    print(item, end="")
    output_array.append(item)

 Based on the provided context, it appears that the second wife did not get excited. The text states that "My madness was quickly caught down," suggesting that the speaker's enthusiasm or excitement was short-lived and may have been brought back under control. Without more information, it is difficult to say why the second wife might have gotten excited in the first place or how she reacted to the situation.

In [ ]:
from transformers import MarianMTModel, MarianTokenizer
import torch

# Replace 'your-marianmt-model' with the name or path of a suitable MarianMT model for Urdu to English
model_name_urdu_to_english = 'Helsinki-NLP/opus-mt-en-ur'
tokenizer_urdu_to_english = MarianTokenizer.from_pretrained(model_name_urdu_to_english)
model_urdu_to_english = MarianMTModel.from_pretrained(model_name_urdu_to_english)

# Assuming you have the 'output_array' containing English sentences
urdu_translations = []
for item in output_array:
    print(item)
    # Tokenize and encode the English text
    inputs = tokenizer_urdu_to_english(item, return_tensors="pt", max_length=512, truncation=True)

    # Generate a translation using the Urdu to English MarianMT model
    with torch.no_grad():
        output_translation = model_urdu_to_english.generate(**inputs)

    # Decode and append the generated translation
    urdu_translation = tokenizer_urdu_to_english.decode(output_translation[0], skip_special_tokens=False)
    urdu_translations.append(urdu_translation)

print(urdu_translations)


/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



 Based
 on
 the
 provided
 context
,
 it
 appears
 that
 the
 second
 wife
 did
 not
 get
 excited
.
 The
 text
 states
 that
 "
My
 mad
ness
 was
 quickly
 caught
 down
,"
 suggesting
 that
 the
 speaker
'
s
 enthus
ias
m
 or
 excitement
 was
 short
-
l
ived
 and
 may
 have
 been
 brought
 back
 under
 control
.
 Without
 more
 information
,
 it
 is
 difficult
 to
 say
